### Regression

Regression is a supervised statistical technique that is used to predict continuous quantities from a given sample; it is able to *estimate* the interesting quantity.

It consists of two steps:
- Training set used to fit the model.
- Validation/Test set are used to evaluate the model.

MSE is a measure of errors but the RMSE is useful for our purposes because it shares the same unit of measurement of data. Also $R^2$ can be useful.